In [5]:
!pip3 install pydantic

  Obtaining dependency information for pydantic from https://files.pythonhosted.org/packages/73/66/0a72c9fcde42e5650c8d8d5c5c1873b9a3893018020c77ca8eb62708b923/pydantic-2.4.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.6/158.6 kB 2.6 MB/s eta 0:00:003.0 MB/s eta 0:00:01
  Obtaining dependency information for annotated-types>=0.4.0 from https://files.pythonhosted.org/packages/28/78/d31230046e58c207284c6b2c4e8d96e6d3cb4e52354721b944d3e1ee4aa5/annotated_types-0.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic-core==2.10.1 from https://files.pythonhosted.org/packages/8d/ab/baf66342f1d18228ed3e54a05308b967c3de3692517f3237c570447e6256/pydantic_core-2.10.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.6.1 from https://files.pythonhosted.org/packages/24/21/7d397a4b7934ff4028987914ac1044d3b7d52712f30e2ac7a2ae5bc86dd0/typing_extensions-4.8.0-py3-none-any.whl.metadata
   ━━━━━━━━━━

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from openai import ChatCompletion,Completion

In [3]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

In [4]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [7]:
def add_five(x: int):
    "Adds 5 to x"
    return x + 5

In [8]:
schema(add_five)

/var/folders/p1/dbb45q7j7r1d45_fhzqfhzxw0000gq/T/ipykernel_75759/3240321801.py:4: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


{'name': 'add_five',
 'description': 'Adds 5 to x',
 'parameters': {'properties': {'x': {'title': 'X', 'type': 'integer'}},
  'required': ['x'],
  'title': 'Input for `add_five`',
  'type': 'object'}}

In [19]:
c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": "Use the add_five function call"},
              {"role": "user", "content": "Add 5 to 10"}],
    functions=[schema(add_five)])

/var/folders/p1/dbb45q7j7r1d45_fhzqfhzxw0000gq/T/ipykernel_75759/3240321801.py:4: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [20]:
c.choices[0].message

<OpenAIObject at 0x105fc1e50> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "add_five",
    "arguments": "{\n  \"x\": 10\n}"
  }
}

In [21]:
c.choices[0].message.function_call.arguments

'{\n  "x": 10\n}'

In [22]:
c.usage

<OpenAIObject at 0x105a75af0> JSON: {
  "prompt_tokens": 58,
  "completion_tokens": 15,
  "total_tokens": 73
}